In [1]:
import os
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sagemaker.tensorflow import TensorFlow
import boto3
import tensorflow as tf
from tensorflow import keras

In [3]:
role = get_execution_role()

In [5]:
sagemaker_session = sagemaker.Session()

In [6]:
s3 = sagemaker_session.boto_session.resource('s3')

In [7]:
bucket = 'arn:aws:s3:::sagemaker-pneumonia-bucket'

In [9]:
# !pip install tensorflow -t ./

In [12]:
tf.__version__

'2.3.1'

In [14]:
data_path = 's3://sagemaker-pneumonia-bucket/chest_xray'

local_inputs = {
    'train': '{}/train'.format(data_path),
    'validation': '{}/val'.format(data_path),
    'eval': '{}/test'.format(data_path),
}

In [23]:
local_hyperparams = {'epochs': 1, 'batch-size': 128}

estimator = TensorFlow(entry_point='modeling.py',
                       hyperparameters=local_hyperparams,
                       role=role,
                       framework_version='2.2.0',
                       py_version='py37',
                       instance_count=1,
                       instance_type='ml.m5.xlarge',
                       base_job_name='pneumonia-tf'
                       )

In [25]:
estimator.fit(local_inputs)

2020-12-04 05:11:32 Starting - Starting the training job...
2020-12-04 05:11:35 Starting - Launching requested ML instances......
2020-12-04 05:12:50 Starting - Preparing the instances for training...
2020-12-04 05:13:23 Downloading - Downloading input data.........
2020-12-04 05:14:54 Training - Training image download completed. Training in progress..2020-12-04 05:14:55.375111: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2020-12-04 05:14:55.381665: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2020-12-04 05:14:55.570012: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2020-12-04 05:14:59,676 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2020-12-04 05:14:59,684 sagemaker-training-toolkit IN

In [26]:
classifier = estimator.deploy(initial_instance_count=1,instance_type='ml.m5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!